In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.linalg
import matplotlib.pyplot as plt
import linear_shrinkage
import Util
from os.path import join
from generating_MTP import generate_mat as generate_MTP2
import os

In [2]:
ret, ret_nonan, univ, tradeidx, dates = Util.load_data()

In [4]:
ret.shape

(10344, 3251)

In [3]:
FF = np.load('data/FF_5.npy')

In [4]:
FF_1 = FF[:, 0].reshape(-1, 1)

In [5]:
Ns = [100, 200, 500, 1000]
ratios = [0.25, 0.5, 1, 2, 4, None]
P = 1

In [5]:
for N in Ns:
    for r in ratios:
        if r is None:
            T = 1260
        else:
            T = int(r * N)
        if T >= 2000:
            continue
            #then the array will contain nans
        print("Working on N = {}, T = {}".format(N, T))
        pastRets = []
        pastFacs = []
        uids = []
#         for h in range(len(univ)):
#             pastRet = Util.get_past_period(h, T, N, univ, tradeidx, ret)
#             pastRets.append(pastRet)
#             pastFac = Util.get_past_period_factor(h, T, tradeidx, FF_1)
#             pastFacs.append(pastFac)
#             uids.append(h)
        folder = "historical_N_{}_T_{}".format(N, T)
        if not os.path.isdir('matlab/data/{}'.format(folder)):
            os.mkdir('matlab/data/{}'.format(folder))
        
        #Util.get_equiweight_OOS(folder, N, P, univ, tradeidx, ret)
        #Util.populate_kendall_covs_from_samples(folder, uids, pastRets)
        #Util.run_parallel_MTP2(7, folder, uids)
        #Util.get_MTP2_OOS(folder, N, P, univ, tradeidx, ret)
        
        #Util.run_parallel_LS(7, folder, pastRets, uids)
        #Util.get_LS_OOS(folder, N, P, univ, tradeidx, ret)
        
        #Util.run_parallel_AFM(4, folder, pastRets, uids, pastFacs)
        #Util.get_AFM_OOS(folder, N, P, univ, tradeidx, ret)

Working on N = 100, T = 25
Working on N = 100, T = 50
Working on N = 100, T = 100
Working on N = 100, T = 200
Working on N = 100, T = 400
Working on N = 100, T = 1260
Working on N = 200, T = 50
Working on N = 200, T = 100
Working on N = 200, T = 200
Working on N = 200, T = 400
Working on N = 200, T = 800
Working on N = 200, T = 1260
Working on N = 500, T = 125
Working on N = 500, T = 250
Working on N = 500, T = 500
Working on N = 500, T = 1000
Working on N = 500, T = 1260
Working on N = 1000, T = 250
Working on N = 1000, T = 500
Working on N = 1000, T = 1000
Working on N = 1000, T = 1260


In [56]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

In [ ]:
for N in Ns:
    if N > 1000:
        continue
    if N == "lol":
        f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row', figsize=(8,5))
        axes = [ax1, ax2, ax3, ax4]
    else:
        f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, sharex='col', sharey='row', figsize=(8,8))
        axes = [ax1, ax2, ax3, ax4, ax5, ax6]
    f.suptitle("N = {}".format(N))
    plt.subplots_adjust(wspace = 0.75, hspace = 0.75)
    for r, ax in zip(ratios, axes):
        if r is None:
            T = 1260
        else:
            T = int(r * N)
        if T >= 2000:
            continue
        folder = "historical_N_{}_T_{}".format(N, T)
        MTP2_rets, LS_rets, AFM_rets = Util.load_OOS(folder, AFM = True)
        
        folder = "kendall_cov_" + folder
        kendall_MTP2_rets, kendall_LS_rets, _ = Util.load_OOS(folder)
        #print("N = {}, T = {}".format(N, T))
        #print("MTP2 var {} vs. LS var {}".format(np.var(MTP2_rets), np.var(LS_rets)))
        #ax.set_xlabel("Number of months evaluated")
        #ax.set_ylabel("variance(returns) for given number of months")
        MTP2_std = 100*np.sqrt(12)*float(np.std(MTP2_rets))
        LS_std = 100*np.sqrt(12)*float(np.std(LS_rets))
        AFM_std = 100*np.sqrt(12)*float(np.std(AFM_rets))
        #print(MTP2_std, LS_std, AFM_std)
        kMTP2_std = 100*np.sqrt(12)*float(np.std(kendall_MTP2_rets))
        kLS_std = 100*np.sqrt(12)*float(np.std(kendall_LS_rets))
        
        #ax.set_title("T = {}\nMTP2 std: {:0.2f}%, {:0.2f}% AFM: {:0.2f}%\nLS std: {:0.2f}%, {:0.2f}%\n".format(T, 
        #                                                                   MTP2_std, kMTP2_std, AFM_std,
        #                                                                   LS_std, kLS_std))
        ax.set_title("T = {}\nMTP2 std: {:0.2f}% \nAFM std: {:0.2f}%\nLS std: {:0.2f}%".format(T, 
                                                                                             MTP2_std, 
                                                                                             AFM_std, 
                                                                                             LS_std))
        ax.plot(Util.cumulative_annualized_std(MTP2_rets), label = 'MTP2')
        ax.plot(Util.cumulative_annualized_std(LS_rets), label = 'LS')
        ax.plot(Util.cumulative_annualized_std(kendall_MTP2_rets), label = 'Kendall MTP2')
        ax.plot(Util.cumulative_annualized_std(kendall_LS_rets), label = 'Kendall LS')

    f.savefig('./figures/historical_data_N_{}'.format(N))
    plt.show()
    print("===========")

# Dataframe 

In [1]:
def initialize_df():
    N_t_tuples = []
    for N in Ns:
        for r in ratios:
            if r is None:
                T = 1260
            else:
                T = int(r * N)
            if T >= 2000:
                continue
            if Util.skip_weak(N, T):
                continue
            N_t_tuples.append((N, T))

    idx = pd.MultiIndex.from_tuples(N_t_tuples, names=["N\n(number of assets)", "T\n(lookback period)"])
    col = ['1/N', 'Linear Shrinkage', 'Approximate Factor Model', 'MTP2']
    #col += ['LS Kendall Tau', 'MTP2 Kendall Tau']
    df = pd.DataFrame('-', idx, col)
    return df

In [2]:
df = initialize_df()

NameError: name 'Ns' is not defined

In [36]:
for N in Ns:
    if N > 1000:
        continue
    for r in ratios:
        if r is None:
            T = 1260
        else:
            T = int(r * N)
        if T >= 2000:
            continue
        
        if Util.skip_weak(N, T):
            continue
        folder = "historical_N_{}_T_{}".format(N, T)
        MTP2_rets, LS_rets, AFM_rets = Util.load_OOS(folder, AFM = True)
        equiweight_rets = Util.load_equiweight_OOS(folder)
        
        MTP2_std = Util.get_std(MTP2_rets)
        LS_std = Util.get_std(LS_rets)
        AFM_std = Util.get_std(AFM_rets)
        equiweight_std = Util.get_std(equiweight_rets)
        
        p_vals_AFM = Util.read_hyp_results(folder, 'MTP2', 'AFM', sharpe = False)
        
        afm_p_val = p_vals_AFM.values[0][3]
        addendum = ""
        if afm_p_val <= 0.01:
            addendum += "***"
        elif afm_p_val <= 0.05:
            addendum += "**"
        elif afm_p_val <= 0.1:
            addendum += "*"
        
        #print(afm_p_val)
        df.at[(N,T), '1/N'] = "{0:.3f}".format(equiweight_std)
        df.at[(N, T), 'Linear Shrinkage'] = "{0:.3f}".format(LS_std)
        df.at[(N, T), 'MTP2'] = "{0:.3f}".format(MTP2_std) + addendum if MTP2_std != 0 else '-'
        df.at[(N, T), 'Approximate Factor Model'] = "{0:.3f}".format(AFM_std)
        
        #All the KT stuff that we're ignoring for now
        #folder = "kendall_cov_" + folder
        #kendall_MTP2_rets, kendall_LS_rets, _ = Util.load_OOS(folder)
        #kMTP2_std = Util.get_std(kendall_MTP2_rets)
        #kLS_std = Util.get_std(kendall_LS_rets)
        #df.at[(N, T), 'MTP2 Kendall Tau'] = kMTP2_std if kMTP2_std != 0 else '-'
        #df.at[(N, T), 'LS Kendall Tau'] = kLS_std if kLS_std != 0 else '-'

In [37]:
df

1/N Linear Shrinkage  \
N\n(number of assets) T\n(lookback period)                            
100                   25                    18.724           13.844   
                      50                    18.724           13.452   
                      100                   18.724           13.695   
                      200                   18.724           12.560   
                      400                   18.724           12.451   
                      1260                  18.724           12.151   
200                   50                    18.134           12.836   
                      100                   18.134           12.583   
                      200                   18.134           11.881   
                      400                   18.134           11.656   
                      800                   18.134           11.670   
                      1260                  18.134           11.665   
500                   125                   17.925           11.623   
                      250                   17.925           11.140   
                      500                   17.925           11.934   
                      1000                  17.925           11.373   

                                           Approximate Factor Model    MTP2  
N\n(number of assets) T\n(lookback period)                                   
100                   25                                     13.653  13.286  
                      50                                     13.119  12.655  
                      100                                    13.234  12.327  
                      200                                    12.389  11.858  
                      400                                    12.398  12.294  
                      1260                                   12.133  12.087  
200                   50                                     12.886  12.117  
                      100                                    12.382  11.803  
                      200                                    11.586  11.586  
                      400                                    11.534  11.880  
                      800                                    11.532  11.635  
                      1260                                   11.572  11.710  
500                   125                                    11.577  10.546  
                      250                                    10.677  10.455  
                      500                                    11.109  11.009  
                      1000                                   10.942  11.031

In [40]:
print(df.to_latex())

\begin{tabular}{llllll}
\toprule
    &      &     1/N & Linear Shrinkage & Approximate Factor Model &    MTP2 \\
N
(number of assets) & T
(lookback period) &         &                  &                          &         \\
\midrule
100 & 25   &  18.724 &           13.844 &                   13.653 &  13.286 \\
    & 50   &  18.724 &           13.452 &                   13.119 &  12.655 \\
    & 100  &  18.724 &           13.695 &                   13.234 &  12.327 \\
    & 200  &  18.724 &           12.560 &                   12.389 &  11.858 \\
    & 400  &  18.724 &           12.451 &                   12.398 &  12.294 \\
    & 1260 &  18.724 &           12.151 &                   12.133 &  12.087 \\
200 & 50   &  18.134 &           12.836 &                   12.886 &  12.117 \\
    & 100  &  18.134 &           12.583 &                   12.382 &  11.803 \\
    & 200  &  18.134 &           11.881 &                   11.586 &  11.586 \\
    & 400  &  18.134 &           11.656 &     